In [1]:
from pyCascade import probeSetup

# working_dir = pathlib.Path().absolute()
working_dir = '/scratch/users/nbachand/Cascade/city_block_cfd'
category =  "single_family"
run = 1
Lx = 480
n_stack = int(Lx+1)
Lx_interest = Lx/2


In [2]:

tiles = {}
x_spacings = [1,3,5]

for i in range(len(x_spacings)):
    x_dist = x_spacings[i]*(Lx/2)/6
    tiles[i] = probeSetup.y_col(x_dist,(Lx/2),[0,Lx],n_stack)
    probeSetup.writeProbes(tiles[i], f"{working_dir}/CHARLES/{category}/R{run}/probes/locations/x_{x_spacings[i]}over6.txt")
    
print('done')

done
